In [82]:
print('hello')

hello


In [83]:
import dotenv

# dotenv.load_dotenv()

In [84]:
# from langchain_groq import ChatGroq

# model = ChatGroq(model="qwen/qwen3-32b",temperature=0.7)
# print(model)

In [85]:
# try:
# 	result = model.invoke("What is the capital of France?").content
# 	print(result)
# except Exception as e:
# 	print(f"Error: {e}")
# 	print("Please check your internet connection, API key, and Groq API status.")

In [86]:
from langchain_community.chat_models import ChatOllama

ollama_chat_model = ChatOllama(model="mistral")
print(ollama_chat_model)

model='mistral'


In [87]:
print(ollama_chat_model.invoke("What is the capital of France?").content)

 The capital of France is Paris.


In [88]:
from langchain_community.embeddings import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="nomic-embed-text")
print(embedding_model)

base_url='http://localhost:11434' model='nomic-embed-text' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


In [90]:
txt = embedding_model.embed_query('What is the capital of France?')

In [44]:
print(len(txt))

768


1. Data Ingestion

In [66]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [67]:
import os 
file_path = Path(os.path.join(os.getcwd(), 'data', 'sample.pdf'))
print(file_path)

c:\Users\Suel.Abbasi\Desktop\agentic ai and RAG\Krish Nair\document_portal\notebooks\data\sample.pdf


In [68]:
loader = PyPDFLoader(file_path)
documents = loader.load()

In [69]:
print(len(documents))

77


In [71]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                    chunk_overlap=150,  # experimental value, there is not deterministic way
                    length_function=len)

In [72]:
docs = text_splitter.split_documents(documents)
print(len(docs))

765


In [73]:
docs[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev'

In [76]:
print(docs[100].metadata)

{'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\Suel.Abbasi\\Desktop\\agentic ai and RAG\\Krish Nair\\document_portal\\notebooks\\data\\sample.pdf', 'total_pages': 77, 'page': 10, 'page_label': '11'}


In [89]:
from langchain.vectorstores import FAISS

In [93]:
vectorstores = FAISS.from_documents(docs, embedding_model)

In [94]:
# FAISS is in memory, so it will not be saved to disk.
# To save it to disk, you can use the `save_local` method.
# on disk storage (faiss we cab persist voer the disk like we do in chroma)
# not available on cloud (pinecone, azure  etc)

In [99]:
vectorstores.similarity_search("What is the llama2 llm model")

[Document(id='c0499342-e695-4691-b563-d05bd7e8ffb4', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\Users\\Suel.Abbasi\\Desktop\\agentic ai and RAG\\Krish Nair\\document_portal\\notebooks\\data\\sample.pdf', 'total_pages': 77, 'page': 76, 'page_label': '77'}, page_content='Llama 2’s potential outputs cannot be predicted in advance, and the model may in some instances\nproduce inaccurate or objectionable responses to user prompts. Therefore, before deploying any\napplications ofLlama 2, developers should perform safety testing and tuning tailored to their\nspecific applications of the model. Please see the Responsible Use Guide available available at\nhttps://ai.meta.com/llama/

In [100]:
retriever =vectorstores.as_retriever()

# create a chain 
# prompt template

# context is somethin that we retireve from vectorstore
# question is the user question
prompt_template = """You are a helpful assistant.
You will be provided with a question and some context.
Your task is to answer the question based on the context provided. IF context doesn't have the 
information, then say "I don't know".

Context: {context}
Question: {question}
Answer:"""

In [101]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)   

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template='You are a helpful assistant.\nYou will be provided with a question and some context.\nYour task is to answer the question based on the context provided. IF context doesn\'t have the \ninformation, then say "I don\'t know".\n\nContext: {context}\nQuestion: {question}\nAnswer:'


In [102]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

In [108]:
rag_chain

({'context': VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000157EA586A50>, search_kwargs={})
  | RunnableLambda(format_docs),
  'question': RunnablePassthrough()},
 PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are a helpful assistant.\nYou will be provided with a question and some context.\nYour task is to answer the question based on the context provided. IF context doesn\'t have the \ninformation, then say "I don\'t know".\n\nContext: {context}\nQuestion: {question}\nAnswer:')
 | ChatOllama(model='mistral')
 | StrOutputParser())

In [110]:
# pass this prompt tp llm
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
output_parser = StrOutputParser()

rag_chain = (
    {'context': retriever | format_docs , "question": RunnablePassthrough()})

In [ ]:
chain = rag_chain | prompt | ollama_chat_model | output_parser
response  = chain.invoke("tell me about the llama2 finetuning benchmark experiments within 25 words")
print(response)